In [1]:
! pip install bertopic



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 2.5 MB/s 
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 88 kB 3.2 MB/s 
     |████████████████████████████████| 5.2 MB 42.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.4 MB 41.6 MB/s 
     |████████████████████████████████| 1.2 MB 64.6 MB/s 
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 636 kB 67.0 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
     |████████████████████████████████| 1.1 MB 45.5 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342148 sha256=0d8876a97da78384fba92e2dba8fdfca04d5a944715c06b77b729cdd17ce67c0
  Stored in directory: /root/.cache/

In [2]:

from bertopic import BERTopic
import pandas as pd



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import re
import gensim
from nltk.stem import WordNetLemmatizer

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
   # tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#)', ' ', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    #tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    #tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
  # tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    tweet = re.sub(' amp ', '&', tweet)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    # tweet = remove_users(tweet)
    # tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub(' & ', '', tweet)
    return tweet

def tokenize_tweets(df):
    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [5]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_nlp_com4.csv",  lineterminator='\n')


In [6]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com2 = df1[df1["community"]=="community_2"]
df1_com3 = df1[df1["community"]=="community_3"]
df1_com4 = df1[df1["community"]=="community_4"]


In [7]:
joinDf = [df1_com0, df1_com1, df1_com2, df1_com3, df1_com4]

In [8]:
df = pd.concat(joinDf)

In [9]:
df = df.reset_index(drop=True)

In [10]:
df

,id,conversation_id,created_at,date,time,timezone,user_id,name,place,tweet,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,community,original_community_extension
0,1418283068641656840,1418283068641656832,2021-07-22 19:53:28+01:00,2021-07-22,19:53:28,100,1617917486,Uncensored Truth,NaN,#Biden Seems to Have Created a New Language Fo...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,86
1,1418281838456496131,1418281838456496128,2021-07-22 19:48:35+01:00,2021-07-22,19:48:35,100,1617917486,Uncensored Truth,NaN,"Ted Cruz Mocks #JenPsaki, Castigates #Democrat...",...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,86
2,1418253956741013516,1418253956741013504,2021-07-22 17:57:47+01:00,2021-07-22,17:57:47,100,1617917486,Uncensored Truth,NaN,#Biden admin backtracks after promoting #CRT g...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,86
3,1418252980806160385,1418252980806160384,2021-07-22 17:53:55+01:00,2021-07-22,17:53:55,100,1617917486,Uncensored Truth,NaN,6 of the #TexasDemocrats actually infected som...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,86
4,1454589609795624967,1454540831621275648,2021-10-31 00:22:42+01:00,2021-10-31,00:22:42,100,1617917486,Uncensored Truth,NaN,@POTUS @POTUS Theres no way the #Biden admin w...,...,NaN,NaN,NaN,"[{'screen_name': 'POTUS', 'name': 'President B...",NaN,NaN,NaN,NaN,community_0,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,1440765383191920641,1440752908056739840,2021-09-22 20:50:10+01:00,2021-09-22,20:50:10,100,1242570896,Mark Collins,NaN,@DavePerryCGAI Essentially #Biden is as #Ameri...,...,NaN,NaN,NaN,"[{'screen_name': 'DavePerryCGAI', 'name': 'Dav...",NaN,NaN,NaN,NaN,community_4,46
586,1438193483077070848,1080252168693841920,2021-09-15 18:30:21+01:00,2021-09-15,18:30:21,100,1242570896,Mark Collins,NaN,@JamesKraska @newscomauHQ @JerryHendrixII @Con...,...,NaN,NaN,NaN,"[{'screen_name': 'JamesKraska', 'name': 'James...",NaN,NaN,NaN,NaN,community_4,46
587,1455664859798978564,1455664859798978560,2021-11-02 22:35:22+00:00,2021-11-02,22:35:22,100,1242570896,Mark Collins,NaN,#VirginiaElection2021: @CNNPolitics with @wolf...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_4,46
588,1458174075612848132,1398731013199593472,2021-11-09 20:46:05+00:00,2021-11-09,20:46:05,100,1242570896,Mark Collins,NaN,@nspector4 @GovWhitmer @SecGranholm @WHNSC #Bi...,...,NaN,NaN,NaN,"[{'screen_name': 'nspector4', 'name': 'Norman ...",NaN,NaN,NaN,NaN,community_4,46


In [11]:
df = df.drop_duplicates()

In [12]:
df["tweet"] = df.tweet.astype("str")

In [13]:
import nltk


In [14]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [15]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)


In [25]:
df.value_counts()

Series([], dtype: int64)

In [26]:
df.community.value_counts()

community_0    444
community_2     89
community_3     26
community_4     19
community_1     12
Name: community, dtype: int64

In [27]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()

In [28]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [29]:
topic_model.get_topic_info()


,Topic,Count,Name
0,3,537,3_biden_potus_joebiden_covid
1,1,22,1_put_impudence_generalstrike_tire
2,2,18,2_ethiopia_potus_nomore_administration
3,0,13,0_cuba_cuban_support_place


In [30]:
topic_model.visualize_barchart()


In [31]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)


In [32]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [33]:
 topic_model.get_representative_docs(0)


['biden press secretary jenpsaki say thats real communism talk cuba wtf work',
 'nikkifried potus word meaningless meaningful action cuban government restrict internet access face widespread protest potus potus joebiden joebiden tantamount break promise want action',
 'instagram video bootsriley cuba colombia joebide']

### Topic Modelling su tutti i tweet del quarto semestre

In [34]:
df1 = df1.drop_duplicates()

In [35]:
df1["tweet"] = df1.tweet.astype("str")

In [36]:
df1["preprocessed_tweet"] = df1["tweet"].apply(preprocess_tweet)


In [37]:
tweets = df1.preprocessed_tweet.to_list()
timestamp = df1.date.to_list()

In [38]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [39]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,5766,-1_biden_joebiden_potus_trump
1,0,1303,0_covid_vaccine_vaccinate_biden
2,1,1260,1_afghanistan_taliban_biden_afghan
3,2,355,2_racism_truth_facts_capitolriots
4,3,325,3_iran_israel_yemen_syria
5,4,306,4_china_taiwan_india_chinese
6,5,302,5_joebiden_joebide_thank_president
7,6,294,6_tax_debt_owe_ados
8,7,283,7_oil_gas_price_energy
9,8,280,8_border_illegal_migrants_immigrants


In [40]:
topic_model.visualize_barchart()

In [41]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [42]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

### Topic Modelling prime cinque comunità per numero di tweets

In [43]:
df1['community'].value_counts()

community_0      444
community_39     362
community_78     352
community_571    262
community_156    190
                ... 
community_711      1
community_389      1
community_906      1
community_289      1
community_256      1
Name: community, Length: 977, dtype: int64

In [44]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com39 = df1[df1["community"]=="community_39"]
df1_com78 = df1[df1["community"]=="community_78"]
df1_com571 = df1[df1["community"]=="community_571"]
df1_com156 = df1[df1["community"]=="community_156"]



In [45]:
joinDf = [df1_com0, df1_com39, df1_com78, df1_com571, df1_com156]

In [46]:
df = pd.concat(joinDf)

In [47]:
df = df.reset_index(drop=True)

In [48]:
df = df.drop_duplicates()

In [49]:
df["tweet"] = df.tweet.astype("str")

In [50]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)

In [51]:
df.community.value_counts()

community_0      444
community_39     362
community_78     352
community_571    262
community_156    190
Name: community, dtype: int64

In [52]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()



In [53]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [54]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,475,-1_biden_trump_racism_truth
1,4,182,4_afghanistan_taliban_afghan_joebiden
2,16,171,16_biden_president_bide_watch
3,2,164,2_covid_potus_vaccine_vaccinate
4,22,99,22_china_putin_media_russia
5,25,54,25_arrest_truth_capitolriots_facts
6,10,40,10_oil_energy_source_price
7,13,39,13_democrats_massive_trillion_america
8,28,32,28_paramount_arresttrumpnow_gqpdeathcult_racism
9,7,29,7_brave_page_coronavirus_capitolriots


In [55]:
topic_model.visualize_barchart()

In [56]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [57]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)